<a href="https://colab.research.google.com/github/Sesandi2003/parallel-programming/blob/main/parallel_prog_lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!lscpu | grep 'CPU(s):'

CPU(s):                                  2
NUMA node0 CPU(s):                       0,1


In [ ]:
%%writefile serial.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define ARRAY_SIZE 1000000000

int main() {
    // 1. Define and allocate the array (Heap allocation)
    int *arr = (int*) malloc(ARRAY_SIZE * sizeof(int));
    if (arr == NULL) {
        printf("Memory allocation failed!\n");
        return 1;
    }

    // 2. Populate the array (0 to 999,999,999)
    // Note: Filling 1 billion items takes time too!
    for (long i = 0; i < ARRAY_SIZE; i++) {
        arr[i] = i;
    }

    printf("Array populated. Starting serial summation...\n");

    // 3. Measure time consumption
    clock_t start = clock();

    // 4. Calculate the sum serially
    long long total_sum = 0;
    for (long i = 0; i < ARRAY_SIZE; i++) {
        total_sum += arr[i];
    }

    clock_t end = clock();
    double time_spent = (double)(end - start) / CLOCKS_PER_SEC;

    printf("Serial Sum: %lld\n", total_sum);
    printf("Time taken: %f seconds\n", time_spent);

    // Free memory
    free(arr);
    return 0;
}

Overwriting serial.c


In [ ]:
!gcc -o serial serial.c
!./serial

Array populated. Starting serial summation...
Serial Sum: 499999999500000000
Time taken: 3.715323 seconds


In [ ]:
%%writefile parallel_manual.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h> // Header for OpenMP
#include <time.h>

#define ARRAY_SIZE 1000000000
#define NUM_THREADS 4

int main() {
    int *arr = (int*) malloc(ARRAY_SIZE * sizeof(int));
    if (arr == NULL) return 1;

    // Populate array
    #pragma omp parallel for num_threads(NUM_THREADS)
    for (long i = 0; i < ARRAY_SIZE; i++) {
        arr[i] = i;
    }

    printf("Array populated. Starting parallel summation (Manual)...\n");

    long long local_sums[NUM_THREADS] = {0}; // Array to store partial sums
    double start_time = omp_get_wtime(); // OpenMP wall clock time

    // Start Parallel Region
    #pragma omp parallel num_threads(NUM_THREADS)
    {
        int thread_id = omp_get_thread_num();
        long long my_sum = 0; // Local variable for this thread

        // Parallelize the loop. Each thread gets a chunk of the loop.
        #pragma omp for
        for (long i = 0; i < ARRAY_SIZE; i++) {
            my_sum += arr[i];
        }

        // Store local sum
        local_sums[thread_id] = my_sum;
    }

    // Aggregate local sums (Main thread does this)
    long long final_sum = 0;
    for (int i = 0; i < NUM_THREADS; i++) {
        final_sum += local_sums[i];
    }

    double end_time = omp_get_wtime();

    printf("Manual Parallel Sum: %lld\n", final_sum);
    printf("Time taken: %f seconds\n", end_time - start_time);

    free(arr);
    return 0;
}

Overwriting parallel_manual.c


In [ ]:
!gcc -o parallel_manual parallel_manual.c -fopenmp
!./parallel_manual

Array populated. Starting parallel summation (Manual)...
Manual Parallel Sum: 499999999500000000
Time taken: 1.943055 seconds


In [ ]:
%%writefile parallel_reduction.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define ARRAY_SIZE 1000000000
#define NUM_THREADS 4

int main() {
    int *arr = (int*) malloc(ARRAY_SIZE * sizeof(int));
    if (arr == NULL) return 1;

    #pragma omp parallel for num_threads(NUM_THREADS)
    for (long i = 0; i < ARRAY_SIZE; i++) {
        arr[i] = i;
    }

    printf("Starting parallel summation (Reduction)...\n");

    long long total_sum = 0;
    double start_time = omp_get_wtime();

    // The Reduction Clause does the magic
    #pragma omp parallel for reduction(+:total_sum) num_threads(NUM_THREADS)
    for (long i = 0; i < ARRAY_SIZE; i++) {
        total_sum += arr[i];
    }

    double end_time = omp_get_wtime();

    printf("Reduction Sum: %lld\n", total_sum);
    printf("Time taken: %f seconds\n", end_time - start_time);

    free(arr);
    return 0;
}

Overwriting parallel_reduction.c


In [ ]:
!gcc -o parallel_reduction parallel_reduction.c -fopenmp
!./parallel_reduction

Starting parallel summation (Reduction)...
Reduction Sum: 499999999500000000
Time taken: 2.554110 seconds


In [ ]:
%%writefile parallel_reduction_dynamic.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define ARRAY_SIZE 1000000000
#define NUM_THREADS 4

int main() {
    int *arr = (int*) malloc(ARRAY_SIZE * sizeof(int));
    if (arr == NULL) return 1;

    #pragma omp parallel for num_threads(NUM_THREADS)
    for (long i = 0; i < ARRAY_SIZE; i++) {
        arr[i] = i;
    }

    printf("Starting parallel summation (Reduction) with dynamic scheduling...\n");

    long long total_sum = 0;
    double start_time = omp_get_wtime();

    // The Reduction Clause does the magic
    #pragma omp parallel for reduction(+:total_sum) schedule(dynamic, 10000) num_threads(NUM_THREADS)
    for (long i = 0; i < ARRAY_SIZE; i++) {
        total_sum += arr[i];
    }

    double end_time = omp_get_wtime();

    printf("Reduction Sum: %lld\n", total_sum);
    printf("Time taken: %f seconds\n", end_time - start_time);

    free(arr);
    return 0;
}

Overwriting parallel_reduction_dynamic.c


In [ ]:
!gcc -o parallel_reduction_dynamic parallel_reduction_dynamic.c -fopenmp
!./parallel_reduction_dynamic

Starting parallel summation (Reduction) with dynamic scheduling...
Reduction Sum: 499999999500000000
Time taken: 1.993117 seconds
